In [1]:
%reset -f

In [16]:
#.\.venv\Scripts\Activate.ps1

import pandas as pd
import numpy as np
from pathlib import Path

bases_path = Path("bases")

checklist = bases_path / "HERING_FRANQUIA_checklist_RA_RA_FRANQUIA_2025-12-17.xlsx"
cancelado = bases_path / "CA_CANCELADO_17-12.xlsx"
relatorio = bases_path / "relatorio-full-lw.xlsx"
df_check = pd.read_excel(checklist, engine="calamine", sheet_name="CHECKLIST", usecols=["TRANSITO", "FILIAL", "EMBALADO", "RESERVADO", "VELOCIDADE_VENDA", "LOJA", "PRODUTO", "COR", "TAM", "RESSUPRIR", "REGIAO", "DATA"])
df_cancelado = pd.read_excel(cancelado, engine="calamine")
df_relatorio = pd.read_excel(relatorio, header=5, engine="calamine", usecols=["Artigo", "Cor", "Tamanho", "Atendido (SAP)", "Cod. Loja" ])




In [17]:
df_check['DATA']

0         2025-12-17
1         2025-12-17
2         2025-12-17
3         2025-12-17
4         2025-12-17
             ...    
371049    2025-12-17
371050    2025-12-17
371051    2025-12-17
371052    2025-12-17
371053    2025-12-17
Name: DATA, Length: 371054, dtype: object

In [18]:
# Itera sobre o nome de cada coluna no seu DataFrame
for coluna in df_check.columns:
    # Verifica se a coluna é do tipo 'object', que é como o pandas armazena strings
    if df_check[coluna].dtype == 'object':
        print(f"Limpando espaços da coluna: {coluna}")
        # Se for uma coluna de texto, remove os espaços em branco do início e do fim
        df_check[coluna] = df_check[coluna].str.strip()

print("\nColunas de texto foram limpas.")

Limpando espaços da coluna: REGIAO
Limpando espaços da coluna: PRODUTO
Limpando espaços da coluna: COR
Limpando espaços da coluna: TAM
Limpando espaços da coluna: DATA

Colunas de texto foram limpas.


In [19]:
df_check.columns

Index(['FILIAL', 'REGIAO', 'PRODUTO', 'COR', 'TAM', 'VELOCIDADE_VENDA', 'LOJA',
       'TRANSITO', 'EMBALADO', 'RESERVADO', 'RESSUPRIR', 'DATA'],
      dtype='object')

In [20]:
df_output = (df_check.groupby(['FILIAL'])[['VELOCIDADE_VENDA']].sum().query('VELOCIDADE_VENDA <= 0').reset_index())
df_output.head()

,FILIAL,VELOCIDADE_VENDA
0,1159,0.0
1,1292,0.0


In [21]:
df_output_dois = (df_check.groupby(['FILIAL'])[['LOJA']].sum().query('LOJA < 0').reset_index())
df_output_dois.head()

,FILIAL,LOJA


In [22]:
df_cancelado.columns

Index(['PontoVda', 'Sit', 'Usuario', 'Artigo', 'Produtos RA', 'DtCancel',
       'Tamanho', 'Pecas'],
      dtype='object')

In [23]:
df_relatorio.columns

Index(['Cod. Loja', 'Artigo', 'Cor', 'Tamanho', 'Atendido (SAP)'], dtype='object')

In [24]:
df_relatorio.head()

,Cod. Loja,Artigo,Cor,Tamanho,Atendido (SAP)
0,1004,0111,AX7EN,XXG,1
1,1004,0111,M2H07S,P,2
2,1004,0111,MD307S,P,1
3,1004,0111,N0A00S,G,7
4,1004,0111,N0A00S,M,7


In [25]:
df_relatorio.columns

Index(['Cod. Loja', 'Artigo', 'Cor', 'Tamanho', 'Atendido (SAP)'], dtype='object')

In [26]:
df_relatorio["Atendido (SAP)"].sum()

np.int64(5239)

In [27]:
df_relatorio["key"] = df_relatorio["Cod. Loja"].astype(str) + "_" + df_relatorio["Artigo"].astype(str) + "_" + df_relatorio["Cor"].astype(str) + "_" + df_relatorio["Tamanho"].astype(str)

In [28]:
colunas_relatorio = ['key','Atendido (SAP)']

convert_dict = {'key': str,
                'Atendido (SAP)': np.int64}
df_relatorio = df_relatorio[colunas_relatorio]

df_relatorio = df_relatorio.astype(convert_dict)

df_relatorio.head()

,key,Atendido (SAP)
0,1004_0111_AX7EN_XXG,1
1,1004_0111_M2H07S_P,2
2,1004_0111_MD307S_P,1
3,1004_0111_N0A00S_G,7
4,1004_0111_N0A00S_M,7


In [29]:
df_relatorio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   key             2380 non-null   object
 1   Atendido (SAP)  2380 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.3+ KB


In [30]:
df_relatorio = df_relatorio.groupby('key')['Atendido (SAP)'].sum().reset_index()

In [31]:
df_relatorio

,key,Atendido (SAP)
0,1004_0111_AX7EN_XXG,1
1,1004_0111_M2H07S_P,2
2,1004_0111_MD307S_P,1
3,1004_0111_N0A00S_G,7
4,1004_0111_N0A00S_M,7
...,...,...
2375,87_N1TE_N1007S_G,1
2376,87_N3A7_MD3EN_XXG,1
2377,87_N3A7_N0A00S_P,0
2378,87_N3A7_NATEN_P,2


In [32]:
df_relatorio['Atendido (SAP)'].sum()

np.int64(5239)

In [33]:
df_check["key"] = df_check["FILIAL"].astype(str) + "_" + df_check["PRODUTO"].astype(str) + "_" + df_check["COR"].astype(str) + "_" + df_check["TAM"].astype(str)

In [34]:
filtro_região = df_check['REGIAO'].isin(['NORTE', 'NORDESTE'])

filtro_ressuprir = df_check['RESSUPRIR'] > 0 

df_check_filtrado = df_check[filtro_região & filtro_ressuprir]

df_resultado = df_check_filtrado.groupby(['FILIAL','key'])[['EMBALADO', 'RESERVADO', 'TRANSITO']].sum().reset_index()

In [35]:
df_cancelado.columns

Index(['PontoVda', 'Sit', 'Usuario', 'Artigo', 'Produtos RA', 'DtCancel',
       'Tamanho', 'Pecas'],
      dtype='object')

In [36]:
print("--- ANTES DA LIMPEZA ---")
df_cancelado.info()

# 1. Limpa os nomes das colunas, removendo espaços extras
df_cancelado.columns = df_cancelado.columns.str.strip()

# 2. Converte 'DtCancel' para o tipo data
#    'errors='coerce'' trata erros, transformando-os em NaT (Not a Time)
df_cancelado['DtCancel'] = pd.to_datetime(df_cancelado['DtCancel'], errors='coerce')

# 3. (Opcional mas recomendado) Converte a coluna 'Pecas' para o tipo 'Int64'
#    Isso a mantém como número inteiro, mesmo que existam valores nulos (NaN).
df_cancelado['Pecas'] = df_cancelado['Pecas'].astype('Int64')


print("\n\n--- DEPOIS DA LIMPEZA ---")
# O método .info() é excelente para verificar os nomes das colunas e seus tipos
df_cancelado.info()

--- ANTES DA LIMPEZA ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PontoVda     1004 non-null   float64
 1   Sit          1674 non-null   object 
 2   Usuario      1674 non-null   object 
 3   Artigo       1674 non-null   object 
 4   Produtos RA  1674 non-null   object 
 5   DtCancel     1674 non-null   object 
 6   Tamanho      1674 non-null   object 
 7   Pecas        1674 non-null   int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 104.8+ KB


--- DEPOIS DA LIMPEZA ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   PontoVda     1004 non-null   float64       
 1   Sit          1674 non-null   object        
 2   Usuario      1674 non-null   object        
 3   A

In [37]:
df_cancelado.columns

Index(['PontoVda', 'Sit', 'Usuario', 'Artigo', 'Produtos RA', 'DtCancel',
       'Tamanho', 'Pecas'],
      dtype='object')

In [38]:
df_cancelado["key"] = df_cancelado["PontoVda"].astype(str) + "_" + df_cancelado["Artigo"].astype(str) + "_" + df_cancelado["Tamanho"].astype(str).astype(str)

df_cancelado = df_cancelado.groupby('key')['Pecas'].sum().reset_index()

In [39]:
resultado_final = df_resultado.merge(df_relatorio, how='left', on='key')
resultado_final['Atendido (SAP)'] = resultado_final['Atendido (SAP)'].fillna(0).astype(np.int64)

resultado_final

,FILIAL,key,EMBALADO,RESERVADO,TRANSITO,Atendido (SAP)
0,109,109_0111_M2H07S_XG,0,0,1,0
1,109,109_0111_MD307S_P,0,0,0,0
2,109,109_0201_AX7EN_G,0,0,4,0
3,109,109_0201_AX7EN_M,0,0,10,0
4,109,109_0201_AX7EN_P,0,0,3,0
...,...,...,...,...,...,...
7362,1285,1285_KZHA_1BSN_P,0,0,0,0
7363,1285,1285_N1TE_M2H07S_M,0,0,2,0
7364,1285,1285_N1TE_N0A00S_M,0,0,0,0
7365,1285,1285_N3A7_AX7EN_G,0,0,0,0


In [40]:
resultado_final = resultado_final.merge(df_cancelado, how='left', on='key')
resultado_final['Pecas'] = resultado_final['Pecas'].fillna(0).astype(np.int64)
resultado_final

,FILIAL,key,EMBALADO,RESERVADO,TRANSITO,Atendido (SAP),Pecas
0,109,109_0111_M2H07S_XG,0,0,1,0,0
1,109,109_0111_MD307S_P,0,0,0,0,0
2,109,109_0201_AX7EN_G,0,0,4,0,0
3,109,109_0201_AX7EN_M,0,0,10,0,0
4,109,109_0201_AX7EN_P,0,0,3,0,0
...,...,...,...,...,...,...,...
7362,1285,1285_KZHA_1BSN_P,0,0,0,0,0
7363,1285,1285_N1TE_M2H07S_M,0,0,2,0,0
7364,1285,1285_N1TE_N0A00S_M,0,0,0,0,0
7365,1285,1285_N3A7_AX7EN_G,0,0,0,0,0


In [41]:
resultado_final["verificação"] = np.where(
    (resultado_final['EMBALADO'] + 
     resultado_final['RESERVADO'] +
       resultado_final['TRANSITO']
    )
    >= resultado_final['Atendido (SAP)'], 'OK', 'VERIFICAR'
)

In [42]:
resultado_final

,FILIAL,key,EMBALADO,RESERVADO,TRANSITO,Atendido (SAP),Pecas,verificação
0,109,109_0111_M2H07S_XG,0,0,1,0,0,OK
1,109,109_0111_MD307S_P,0,0,0,0,0,OK
2,109,109_0201_AX7EN_G,0,0,4,0,0,OK
3,109,109_0201_AX7EN_M,0,0,10,0,0,OK
4,109,109_0201_AX7EN_P,0,0,3,0,0,OK
...,...,...,...,...,...,...,...,...
7362,1285,1285_KZHA_1BSN_P,0,0,0,0,0,OK
7363,1285,1285_N1TE_M2H07S_M,0,0,2,0,0,OK
7364,1285,1285_N1TE_N0A00S_M,0,0,0,0,0,OK
7365,1285,1285_N3A7_AX7EN_G,0,0,0,0,0,OK


In [43]:
resultado_final_filtrado = resultado_final[resultado_final['verificação'].isin(['VERIFICAR'])]

resultado_final_filtrado

,FILIAL,key,EMBALADO,RESERVADO,TRANSITO,Atendido (SAP),Pecas,verificação


In [44]:
colunas_para_somar = ['EMBALADO', 'RESERVADO', 'TRANSITO', 'Atendido (SAP)', 'Pecas']

# Realizando o Group By e a soma
df_agrupado = resultado_final_filtrado.groupby('FILIAL')[colunas_para_somar].sum().reset_index()

# Renomeando a coluna 'Pecas' para 'Pecas Cancelado' conforme você pediu
df_agrupado = df_agrupado.rename(columns={'Pecas': 'Pecas Cancelado'})

# Visualizar o resultado
df_agrupado

,FILIAL,EMBALADO,RESERVADO,TRANSITO,Atendido (SAP),Pecas Cancelado
